# Notebook to rename oif files

## Imports

In [42]:
import os
from pathlib import Path
import tkinter as tk
from tkinter import filedialog, simpledialog
from showinfm import show_in_file_manager

## Testing

.oif files are stored with an acompanying folder of the same name with .files appended. The file stores metadata while the folder stores the raw image data.  
When renaming an .oif file and .oif.files folder, there is metadata in the file that needs to be updated to refer to the new folder name 

In [43]:
def select_file():
    root = tk.Tk()
    root.withdraw()
    root.attributes("-topmost", 1)
    file_nm = filedialog.askopenfilename(title="Select files")
    if file_nm == "":
        return None
    else:
        return Path(file_nm)

In [44]:
def get_new_name(old_name):
    root = tk.Tk()
    root.withdraw()
    root.attributes("-topmost", 1)
    pad = " " * 50
    prompt_text = "Enter new name of .oif file"
    new_name = simpledialog.askstring("Input", pad+prompt_text+pad, initialvalue=old_name)
    return new_name

In [45]:
def rename_oif_file(old_oif_path, new_name):
    # Set variables
    old_oif_files_path = old_oif_path.with_suffix(".oif.files")

    new_oif_path = old_oif_path.with_name(new_name).with_suffix(".oif")
    new_oif_files_path = old_oif_files_path.with_name(new_name).with_suffix(".oif.files")

    old_name = old_oif_path.stem

    # Check if path to file exists
    if not (os.path.exists(old_oif_path) or os.path.exists(old_oif_files_path)):
        if not os.path.exists(old_oif_path) and os.path.exists(old_oif_files_path):
            error_text = f".oif path does not exist:\n{old_oif_path}"
        elif os.path.exists(old_oif_path) and not os.path.exists(old_oif_files_path):
            error_text = f".oif.files path does not exist:\n{old_oif_files_path}"
        else:
            error_text = f".oif and .oif.files paths do not exist:\n{old_oif_path}\n{old_oif_files_path}"
        raise RuntimeError(error_text)

    # Check if new name is valid
    if (os.path.exists(new_oif_path) or
        os.path.exists(new_oif_files_path)):
        raise RuntimeError("")

    ## update .oif metadata
    # Read file
    with open(old_oif_path, "r", encoding="utf16") as oif:
        data = oif.readlines()
    # Edit data
    for index, line in enumerate(data):
        if old_name in line:
            data[index] = line.replace(old_name, new_name)
    # Write data to file
    with open(old_oif_path, "w", encoding="utf16") as oif:
        oif.writelines(data)

    ## Rename .oif.files
    os.rename(old_oif_path, new_oif_path)
    os.rename(old_oif_files_path, new_oif_files_path)



In [46]:
# Test file
# TEST_FILE = Path("C:/Users/ahshenas/Desktop/rename oif test/20230315_HD9xKaiR1D-crispr_test_name.oif")
def main():
    file_nm = select_file()
    if file_nm is None:
        print("Canceled file selection")
        return
    new_name = get_new_name(file_nm.stem)
    if new_name is None:
        print("Canceled new name selection")
        return
    
    rename_oif_file(file_nm, new_name)

    show_in_file_manager(str(file_nm.parent))

In [47]:
main()